In [2]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import numpy as np

In [3]:
key_path = key_path
project = project_id

# Sources
table = 'bronze_ticker_info'
dataset = 'bronze'
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'silver_ticker_info'
dataset_to_save = 'silver'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'


In [4]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [5]:
df = bigquery.run_query(
    f"""
    SELECT
        ticker,
        shortName,
        industry,
        sector,
        recommendationKey,
        fullTimeEmployees,
        marketCap,
        enterpriseValue,
        floatShares,
        sharesOutstanding,
        sharesShort,
        shortRatio,
        bookValue,
        currentPrice,
        totalCash,
        totalCashPerShare,
        totalDebt,
        totalRevenue,
        revenuePerShare,
        returnOnAssets,
        returnOnEquity,
        operatingCashflow,
        operatingMargins
    FROM {project}.{dataset}.{table}
    """
)
df

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
0,AGTI,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FSR,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GRPH,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GTXI,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ICVX,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10932,KIND,"Nextdoor Holdings, Inc.",Internet Content & Information,Communication Services,hold,594.0,9.364732e+08,5.797210e+08,172789944.0,187940992.0,...,2.465,456540992.0,1.202,44701000.0,228087008.0,0.589,-0.16791,-0.28366,-52211000.0,-0.77444
10933,MAX,"MediaAlpha, Inc.",Internet Content & Information,Communication Services,buy,137.0,1.174837e+09,1.056066e+09,28674897.0,46228000.0,...,17.905,28659000.0,0.524,166852000.0,496670016.0,10.178,-0.00290,NaN,27177000.0,0.04041
10934,NRDS,"NerdWallet, Inc.",Internet Content & Information,Communication Services,buy,730.0,1.008633e+09,8.770533e+08,39056888.0,47021600.0,...,12.815,113800000.0,1.446,7800000.0,599000000.0,7.749,0.00613,-0.02982,98600000.0,-0.06375
10935,OB,Outbrain Inc.,Internet Content & Information,Communication Services,buy,942.0,2.531927e+08,2.236463e+08,29062286.0,49211400.0,...,5.145,162672000.0,3.306,134848000.0,909356032.0,18.267,-0.00158,-0.01227,44633000.0,-0.02628


In [6]:
tickers_indicators = bigquery.run_query(
    f"""
    SELECT DISTINCT
        ticker
    FROM {project}.silver.silver_indicators
    """
)
tickers_indicators

,ticker
0,ECL
1,EG
2,LLY
3,FSLR
4,BAC
...,...
3440,FTC
3441,PKW
3442,SUSL
3443,CBT


In [7]:
# Hacemos un INNER para traernos solo los ticker de los que tenemos historico

df_filtered = pd.merge(df, tickers_indicators, how='inner', on='ticker')
df_filtered = df_filtered[df.columns]
df_filtered

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
0,EU,enCore Energy Corp.,Uranium,Energy,none,NaN,6.816537e+08,6.517439e+08,177623981.0,184730000.0,...,3.690,71774672.0,0.395,2.117133e+07,3.571426e+07,0.2160,-0.06477,-0.06919,-46313972.0,-1.67695
1,EU,enCore Energy Corp.,Uranium,Energy,none,NaN,6.604097e+08,6.517439e+08,177623981.0,184730000.0,...,3.575,71774672.0,0.395,2.117133e+07,3.571426e+07,0.2160,-0.06477,-0.06919,-46313972.0,-1.67695
2,EU,enCore Energy Corp.,Uranium,Energy,none,NaN,6.631807e+08,6.517439e+08,177623981.0,184730000.0,...,3.590,71774672.0,0.395,2.117133e+07,3.571426e+07,0.2160,-0.06477,-0.06919,-46313972.0,-1.67695
3,EU,enCore Energy Corp.,Uranium,Energy,none,NaN,6.751882e+08,6.517439e+08,177623981.0,184730000.0,...,3.655,71774672.0,0.395,2.117133e+07,3.571426e+07,0.2160,-0.06477,-0.06919,-46313972.0,-1.67695
4,EP,Empire Petroleum Corporation,Oil & Gas E&P,Energy,none,50.0,1.716573e+08,1.677530e+08,23279220.0,31381600.0,...,5.470,9257773.0,0.295,1.068807e+07,4.287139e+07,1.6780,-0.08830,-0.41332,203709.0,-0.16408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5751,CTLT,"Catalent, Inc.",Drug Manufacturers—Specialty & Generic,Healthcare,hold,18000.0,1.076469e+10,1.556915e+10,179854154.0,180980000.0,...,59.480,162000000.0,0.895,4.990000e+09,4.135000e+09,22.7510,-0.00712,-0.28290,250000000.0,0.00559
5752,HCM,HUTCHMED (China) Limited,Drug Manufacturers—Specialty & Generic,Healthcare,buy,1970.0,2.992986e+09,1.396775e+10,502034794.0,170888000.0,...,17.300,803510016.0,0.940,8.859400e+07,6.108060e+08,3.5750,-0.05902,-0.05439,-46977000.0,-0.09003
5753,AQST,"Aquestive Therapeutics, Inc.",Drug Manufacturers—Specialty & Generic,Healthcare,buy,135.0,4.006631e+08,3.510689e+08,79978698.0,91059800.0,...,4.400,89870000.0,0.987,3.572300e+07,5.836000e+07,0.7870,-0.10875,NaN,-24679000.0,0.00274
5754,EVO,Evotec SE,Drug Manufacturers—Specialty & Generic,Healthcare,buy,5022.0,1.351537e+09,7.987532e+08,148998744.0,355107008.0,...,3.806,298865984.0,1.686,4.735110e+08,7.884410e+08,2.2265,0.00968,-0.15618,-54905000.0,-0.19850


In [8]:
# Nos quedamos con solo una fila por ticker

# Reemplazar valores NaN en la columna 'market' con 0
df_filtered['marketCap'].fillna(0, inplace=True)

# Encontrar el índice de la fila con el valor más alto en 'market' para cada ticker
idx = df_filtered.groupby('ticker')['marketCap'].idxmax()

# Seleccionar esas filas del DataFrame original
df_filtered = df_filtered.loc[idx]

# Eliminamos los tickers que el marketcap sea 0
df_filtered = df_filtered[df_filtered['marketCap'] != 0]

df_filtered

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
5466,A,"Agilent Technologies, Inc.",Diagnostics & Research,Healthcare,buy,18000.0,4.037208e+10,4.154804e+10,286385268.0,287529984.0,...,140.4100,1.779000e+09,6.187,2.932000e+09,6.497000e+09,22.269,0.08906,0.24658,1.786000e+09,0.21103
2320,AACG,ATA Creativity Global,Education & Training Services,Consumer Defensive,buy,581.0,1.855656e+07,3.715915e+07,18043830.0,31950000.0,...,0.5808,4.010650e+07,0.637,3.889986e+07,2.429485e+08,7.698,-0.05531,-0.35138,NaN,-0.40102
9,AADI,"Aadi Bioscience, Inc.",Biotechnology,Healthcare,buy,70.0,4.110918e+07,-3.443604e+07,16148562.0,24614800.0,...,1.6701,7.856800e+07,3.192,1.056000e+06,2.381700e+07,0.883,-0.34932,-0.62264,-5.040500e+07,-2.52209
5011,AAL,"American Airlines Group, Inc.",Airlines,Industrials,hold,137400.0,6.583457e+09,3.716495e+10,647910648.0,656704000.0,...,10.0250,8.987000e+09,13.685,3.944700e+10,5.344800e+10,81.548,0.02779,NaN,2.015000e+09,0.09823
5308,AAME,Atlantic American Corporation,Insurance—Life,Financial Services,none,154.0,3.212969e+07,5.160161e+07,4055676.0,20399800.0,...,1.5750,2.121300e+07,1.040,4.012000e+07,1.860010e+08,9.117,-0.00146,-0.03087,2.615000e+06,0.00038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,ZUO,"Zuora, Inc.",Software—Infrastructure,Technology,buy,1618.0,1.343303e+09,1.225365e+09,139135713.0,139748992.0,...,9.0700,5.434710e+08,3.638,4.065180e+08,4.456830e+08,3.064,-0.02464,-0.31016,5.554000e+06,-0.07960
2999,ZVRA,"Zevra Therapeutics, Inc.",Biotechnology,Healthcare,buy,65.0,3.772959e+08,3.822178e+08,49294449.0,52617800.0,...,7.1705,4.925800e+07,0.936,5.946800e+07,2.368900e+07,0.603,-0.37412,-1.32319,-5.606000e+07,-5.34839
4049,ZWS,Zurn Elkay Water Solutions Corp,Pollution & Treatment Controls,Industrials,buy,2400.0,5.456876e+09,5.867595e+09,143434647.0,170928000.0,...,31.9250,1.627000e+08,0.952,5.452000e+08,1.541000e+09,8.908,0.05455,0.08043,3.007000e+08,0.17354
3367,ZYME,Zymeworks Inc.,Biotechnology,Healthcare,buy,290.0,8.177556e+08,5.098289e+08,39446221.0,71047400.0,...,11.5100,3.301460e+08,4.647,2.008800e+07,6.270500e+07,0.853,-0.13622,-0.26315,-9.137200e+07,-1.33030


In [9]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3100 entries, 5466 to 3472
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             3100 non-null   object 
 1   shortName          3100 non-null   object 
 2   industry           3097 non-null   object 
 3   sector             3097 non-null   object 
 4   recommendationKey  3098 non-null   object 
 5   fullTimeEmployees  2953 non-null   float64
 6   marketCap          3100 non-null   float64
 7   enterpriseValue    3080 non-null   float64
 8   floatShares        3060 non-null   float64
 9   sharesOutstanding  3091 non-null   float64
 10  sharesShort        3096 non-null   float64
 11  shortRatio         3092 non-null   float64
 12  bookValue          3095 non-null   float64
 13  currentPrice       3098 non-null   float64
 14  totalCash          3081 non-null   float64
 15  totalCashPerShare  3075 non-null   float64
 16  totalDebt          3005 no

In [10]:
# Seleccionar solo las columnas numéricas
numerical_columns = df_filtered.select_dtypes(include=[np.number]).columns

# Identificar columnas numéricas con valores NaN
columns_with_nans = df_filtered[numerical_columns].columns[df_filtered[numerical_columns].isna().any()]

# Calcular la mediana para cada columna numérica con valores NaN
medianas = df_filtered[columns_with_nans].median()

# Rellenar los valores NaN en las columnas numéricas con la mediana correspondiente
df_filtered[columns_with_nans] = df_filtered[columns_with_nans].fillna(medianas)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3100 entries, 5466 to 3472
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             3100 non-null   object 
 1   shortName          3100 non-null   object 
 2   industry           3097 non-null   object 
 3   sector             3097 non-null   object 
 4   recommendationKey  3098 non-null   object 
 5   fullTimeEmployees  3100 non-null   float64
 6   marketCap          3100 non-null   float64
 7   enterpriseValue    3100 non-null   float64
 8   floatShares        3100 non-null   float64
 9   sharesOutstanding  3100 non-null   float64
 10  sharesShort        3100 non-null   float64
 11  shortRatio         3100 non-null   float64
 12  bookValue          3100 non-null   float64
 13  currentPrice       3100 non-null   float64
 14  totalCash          3100 non-null   float64
 15  totalCashPerShare  3100 non-null   float64
 16  totalDebt          3100 no

In [12]:
# Define sector groupings
sector_groups = {
    'Technology': 'Technology and Communications',
    'Communication Services': 'Technology and Communications',
    'Financial Services': 'Financials and Real Estate',
    'Real Estate': 'Financials and Real Estate',
    'Healthcare': 'Healthcare and Defensive Products',
    'Consumer Defensive': 'Healthcare and Defensive Products',
    'Industrials': 'Industrials and Basic Materials',
    'Basic Materials': 'Industrials and Basic Materials',
    'Consumer Cyclical': 'Consumer, Energy, and Utilities',
    'Energy': 'Consumer, Energy, and Utilities',
    'Utilities': 'Consumer, Energy, and Utilities'
}

# Add new columns to the DataFrame
df['Grouped Sector'] = df['sector'].map(sector_groups)

# Show the updated DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10937 entries, 0 to 10936
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             10937 non-null  object 
 1   shortName          10275 non-null  object 
 2   industry           10070 non-null  object 
 3   sector             10070 non-null  object 
 4   recommendationKey  10159 non-null  object 
 5   fullTimeEmployees  8825 non-null   float64
 6   marketCap          9696 non-null   float64
 7   enterpriseValue    9510 non-null   float64
 8   floatShares        9579 non-null   float64
 9   sharesOutstanding  9633 non-null   float64
 10  sharesShort        9745 non-null   float64
 11  shortRatio         9722 non-null   float64
 12  bookValue          9993 non-null   float64
 13  currentPrice       10159 non-null  float64
 14  totalCash          9840 non-null   float64
 15  totalCashPerShare  9745 non-null   float64
 16  totalDebt          962

In [ ]:
# Define industry groupings
industry_groups = {
    'Software': 'Technology and Communications',
    'Biotechnology': 'Healthcare and Defensive Products',
    'Retail': 'Consumer, Energy, and Utilities',
    'Machinery': 'Industrials and Basic Materials',
    'Insurance': 'Financials and Real Estate',
    'Electric': 'Consumer, Energy, and Utilities',
    'Oil & Gas': 'Consumer, Energy, and Utilities'
}

# Add new columns to the DataFrame
df['Grouped Industry'] = df['Industry'].map(industry_groups)


In [12]:
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='ticker', table=table_conca_to_save, new_df=df_filtered)

    if not df_incremental.empty:
        # Guardamos los datos en Bigquery
        bigquery.save_dataframe(df_incremental, project, dataset_to_save, table_to_save, if_exists='append', schema=None)
        print(f'New records loaded')
    else:
        print('No new records to load.')

# En el caso de no tener datos en Bigquery, guardamos todo el df
except Exception as e:
    bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)
    print('New data persisted')
    print(f'Exception encountered: {e}')

No new records to load.
